<a href="https://colab.research.google.com/github/nicolez9911/colab/blob/main/04_melanoma.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install --upgrade --force-reinstall --no-deps kaggle
from google.colab import files
files.upload()

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 83.6/83.6 kB 2.6 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for kaggle: filename=kaggle-1.5.16-py3-none-any.whl size=110683 sha256=cfbc83d7641f97b0573e9f8282affc2b1484d555530893ce897731745610c951
  Stored in directory: /root/.cache/pip/wheels/43/4b/fb/736478af5e8004810081a06259f9aa2f7c3329fc5d03c2c412
Successfully built kaggle
  Attempting uninstall: kaggle
    Found existing installation: kaggle 1.5.16
    Uninstalling kaggle-1.5.16:
      Successfully uninstalled kaggle-1.5.16


Saving kaggle.json to kaggle.json


{'kaggle.json': b'{"username":"nicolez9911","key":"439a3da38e3dab16713f10b91119c5cf"}'}

In [2]:
!mkdir ~/.kaggle
!cp kaggle.json ~/.kaggle/
!chmod 600 ~/.kaggle/kaggle.json
!ls ~/.kaggle

kaggle.json


In [3]:
!kaggle datasets download -d drscarlat/melanoma

100% 5.24G/5.26G [00:32<00:00, 289MB/s]
100% 5.26G/5.26G [00:32<00:00, 172MB/s]


In [4]:
!unzip -qq /content/melanoma.zip

In [5]:
!ls /content

dermmel  DermMel  kaggle.json  melanoma.zip  sample_data


In [6]:
import matplotlib.pyplot as plt

def generate_plot(history):
  # Plot training history (loss and accuracy)
  plt.figure(figsize=(12, 5))

  # Plot training & validation loss
  plt.subplot(1, 2, 1)
  plt.plot(history.history['loss'], label='Training Loss')
  plt.plot(history.history['val_loss'], label='Validation Loss')
  plt.xlabel('Epochs')
  plt.ylabel('Loss')
  plt.legend()
  plt.title('Training and Validation Loss')

  # Plot training & validation accuracy
  plt.subplot(1, 2, 2)
  plt.plot(history.history['accuracy'], label='Training Accuracy')
  plt.plot(history.history['val_accuracy'], label='Validation Accuracy')
  plt.xlabel('Epochs')
  plt.ylabel('Accuracy')
  plt.legend()
  plt.title('Training and Validation Accuracy')

  plt.tight_layout()
  return plt

## Load Dataset

### Image Info

In [20]:
# import required module
from PIL import Image

# get image
filepath = "/content/dermmel/DermMel/train_sep/Melanoma/AUG_0_10.jpeg"
img = Image.open(filepath)

# get width and height
width = img.width
height = img.height

# display width and height
print("The height of the image is: ", height)
print("The width of the image is: ", width)


The height of the image is:  224
The width of the image is:  224


In [11]:
from tensorflow.keras.utils import image_dataset_from_directory

train_dataset = image_dataset_from_directory(
    "DermMel/train_sep",
    image_size=(450, 600),
    batch_size=32)
validation_dataset = image_dataset_from_directory(
    "DermMel/valid",
    image_size=(450, 600),
    batch_size=32)
test_dataset = image_dataset_from_directory(
    "DermMel/test",
    image_size=(450, 600),
    batch_size=32)

Found 10682 files belonging to 2 classes.
Found 3562 files belonging to 2 classes.
Found 3561 files belonging to 2 classes.


In [30]:
import tensorflow as tf
from tensorflow.keras.preprocessing.image import ImageDataGenerator

common_image_size = (224, 224)

datagen = ImageDataGenerator(
      rescale=1./255,
      zoom_range=0.2,
      fill_mode='nearest',
      horizontal_flip=True,
      vertical_flip=True)

batch_size = 32

train_dataset = datagen.flow_from_directory(
    "DermMel/train_sep",
    target_size=common_image_size,
    batch_size=batch_size,
    class_mode='binary' # or 'categorical' depending on your problem
)
validation_dataset = datagen.flow_from_directory(
    "DermMel/valid",
    target_size=common_image_size,
    batch_size=batch_size,
    class_mode='binary'
)
test_dataset = datagen.flow_from_directory(
    "DermMel/test",
    target_size=common_image_size,
    batch_size=batch_size,
    class_mode='binary'
)

Found 10682 images belonging to 2 classes.
Found 3562 images belonging to 2 classes.
Found 3561 images belonging to 2 classes.


In [8]:
import matplotlib.pyplot as plt
import numpy as np

In [22]:
train_dataset[1][0].shape

(32, 224, 224, 3)

In [ ]:
from matplotlib import pyplot
pyplot.imshow(train_dataset[0][0][10])

In [31]:
from tensorflow import keras
from keras import layers

inputs = keras.Input(shape=(224, 224, 3))
data_augmentation = keras.Sequential([])
x = data_augmentation(inputs)
x = layers.Conv2D(filters=32, kernel_size=3, activation="relu")(x)
x = layers.MaxPooling2D(pool_size=2)(x)
x = layers.Conv2D(filters=64, kernel_size=3, activation="relu")(x)
x = layers.MaxPooling2D(pool_size=2)(x)
x = layers.Conv2D(filters=128, kernel_size=3, activation="relu")(x)
x = layers.MaxPooling2D(pool_size=2)(x)
x = layers.Conv2D(filters=256, kernel_size=3, activation="relu")(x)
x = layers.MaxPooling2D(pool_size=2)(x)
x = layers.Conv2D(filters=256, kernel_size=3, activation="relu")(x)
x = layers.Flatten()(x)
x = layers.Dropout(0.5)(x)
outputs = layers.Dense(1, activation="sigmoid")(x)
model = keras.Model(inputs=inputs, outputs=outputs)
model.compile(loss="binary_crossentropy",
  optimizer="rmsprop",
  metrics=["accuracy"])

In [32]:
model.summary()

Model: "model"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_1 (InputLayer)        [(None, 224, 224, 3)]     0         
                                                                 
 sequential_2 (Sequential)   multiple                  0         
                                                                 
 conv2d (Conv2D)             (None, 222, 222, 32)      896       
                                                                 
 max_pooling2d (MaxPooling2  (None, 111, 111, 32)      0         
 D)                                                              
                                                                 
 conv2d_1 (Conv2D)           (None, 109, 109, 64)      18496     
                                                                 
 max_pooling2d_1 (MaxPoolin  (None, 54, 54, 64)        0         
 g2D)                                                        

In [34]:
history = model.fit(train_dataset, validation_data=validation_dataset, epochs=30, batch_size=256)

test_loss = model.evaluate(test_dataset)
validation_loss = model.evaluate(validation_dataset)

plt = generate_plot(history)
plt.show()

test_loss, validation_loss

Epoch 1/30
224/334 [===================>..........] - ETA: 8:40 - loss: 0.6496 - accuracy: 0.6072

KeyboardInterrupt: ignored

In [39]:
conv_base = keras.applications.vgg16.VGG16(
  weights="imagenet",
  include_top=False)
conv_base.trainable = False

In [40]:
inputs = keras.Input(shape=(224, 224, 3))
data_augmentation = keras.Sequential([])
x = data_augmentation(inputs)
x = keras.applications.vgg16.preprocess_input(x)
x = conv_base(x)
x = layers.MaxPooling2D(pool_size=2)(x)
x = layers.Conv2D(filters=256, kernel_size=3, activation="relu")(x)
x = layers.Flatten()(x)
x = layers.Dropout(0.5)(x)
x = layers.Dense(256)(x)
outputs = layers.Dense(1, activation="sigmoid")(x)
model = keras.Model(inputs, outputs)
model.compile(loss="binary_crossentropy",
optimizer="rmsprop",
metrics=["accuracy"])

In [ ]:
model.compile(loss="binary_crossentropy",
  optimizer="rmsprop",
  metrics=["accuracy"])
#model.summary()

In [42]:
history = model.fit(train_dataset, validation_data=validation_dataset, epochs=10)

test_loss = model.evaluate(test_dataset)
validation_loss = model.evaluate(validation_dataset)

plt = generate_plot(history)
plt.show()

test_loss, validation_loss

Epoch 1/10
 11/334 [..............................] - ETA: 1:42:38 - loss: 2.6573 - accuracy: 0.4801

KeyboardInterrupt: ignored

In [37]:
conv_base.trainable = True
for layer in conv_base.layers[:-4]:
  layer.trainable = False

In [38]:
model.compile(loss="binary_crossentropy",
  optimizer="rmsprop",
  metrics=["accuracy"])
model.summary()

Model: "model_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_3 (InputLayer)        [(None, 224, 224, 3)]     0         
                                                                 
 sequential_3 (Sequential)   multiple                  0         
                                                                 
 tf.__operators__.getitem (  (None, 224, 224, 3)       0         
 SlicingOpLambda)                                                
                                                                 
 tf.nn.bias_add (TFOpLambda  (None, 224, 224, 3)       0         
 )                                                               
                                                                 
 vgg16 (Functional)          (None, None, None, 512)   14714688  
                                                                 
 max_pooling2d_4 (MaxPoolin  (None, 3, 3, 512)         0   

In [ ]:
history = model.fit(train_dataset, validation_data=validation_dataset, epochs=10)

test_loss = model.evaluate(test_dataset)
validation_loss = model.evaluate(validation_dataset)

plt = generate_plot(history)
plt.show()

test_loss, validation_loss

----

## Chat GPT example

In [7]:
from tensorflow.keras.applications import VGG16
from tensorflow.keras import layers
from tensorflow.keras import models
from tensorflow.keras import optimizers

# Load the VGG16 model pre-trained on ImageNet data
base_model = VGG16(weights='imagenet', include_top=False, input_shape=(224, 224, 3))

# Freeze the convolutional layers
for layer in base_model.layers:
    layer.trainable = False

# Create a new model on top of VGG16
model = models.Sequential()
model.add(base_model)
model.add(layers.Flatten())
model.add(layers.Dense(256, activation='relu'))
model.add(layers.Dropout(0.5))
model.add(layers.Dense(1, activation='sigmoid'))  # for binary classification

# Compile the model
model.compile(optimizer=optimizers.RMSprop(lr=1e-4), loss='binary_crossentropy', metrics=['accuracy'])


58889256/58889256 [==============================] - 0s 0us/step


In [9]:
import tensorflow as tf
from tensorflow.keras.preprocessing.image import ImageDataGenerator

batch_size = 32
common_image_size = (224, 224)

train_datagen = ImageDataGenerator(
    rescale=1./255,
    rotation_range=40,
    width_shift_range=0.2,
    height_shift_range=0.2,
    shear_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True,
    fill_mode='nearest'
)

train_generator = train_datagen.flow_from_directory(
    "DermMel/train_sep",
    target_size=common_image_size,
    batch_size=batch_size,
    class_mode='binary'
)

validation_datagen = ImageDataGenerator(rescale=1./255)

validation_generator = validation_datagen.flow_from_directory(
    "DermMel/valid",
    target_size=common_image_size,
    batch_size=batch_size,
    class_mode='binary'
)


Found 10682 images belonging to 2 classes.
Found 3562 images belonging to 2 classes.


In [ ]:
epochs = 10  # adjust as needed

history = model.fit(
    train_generator,
    steps_per_epoch=train_generator.samples // batch_size,
    epochs=epochs,
    validation_data=validation_generator,
    validation_steps=validation_generator.samples // batch_size
)


Epoch 1/10
333/333 [==============================] - 228s 642ms/step - loss: 0.6297 - accuracy: 0.7300 - val_loss: 0.3887 - val_accuracy: 0.8215
Epoch 2/10
333/333 [==============================] - 187s 561ms/step - loss: 0.4676 - accuracy: 0.7841 - val_loss: 0.4442 - val_accuracy: 0.7973
Epoch 3/10


-------

## Cats & Dogs example

In [46]:
from tensorflow.keras.utils import image_dataset_from_directory

train_dataset = image_dataset_from_directory(
    "DermMel/train_sep",
    image_size=(224, 224),
    batch_size=32)
validation_dataset = image_dataset_from_directory(
    "DermMel/valid",
    image_size=(224, 224),
    batch_size=32)
test_dataset = image_dataset_from_directory(
    "DermMel/test",
    image_size=(224, 224),
    batch_size=32)

Found 10682 files belonging to 2 classes.
Found 3562 files belonging to 2 classes.
Found 3561 files belonging to 2 classes.


In [47]:
conv_base = keras.applications.vgg16.VGG16(
  weights="imagenet",
  include_top=False)
conv_base.trainable = False

In [48]:
data_augmentation = keras.Sequential([
  layers.RandomFlip("horizontal"),
  layers.RandomRotation(0.1),
  layers.RandomZoom(0.2),
])
inputs = keras.Input(shape=(224, 224, 3))
x = data_augmentation(inputs)
x = keras.applications.vgg16.preprocess_input(x)
x = conv_base(x)
x = layers.Flatten()(x)
x = layers.Dense(256)(x)
outputs = layers.Dense(1, activation="sigmoid")(x)
model = keras.Model(inputs, outputs)

# compile
model.compile(loss="binary_crossentropy",
optimizer="rmsprop",
metrics=["accuracy"])

In [49]:
history = model.fit(train_dataset, validation_data=validation_dataset, epochs=10)

test_loss = model.evaluate(test_dataset)
validation_loss = model.evaluate(validation_dataset)

plt = generate_plot(history)
plt.show()

test_loss, validation_loss

Epoch 1/10
  1/334 [..............................] - ETA: 2:17:26 - loss: 4.4176 - accuracy: 0.4375

KeyboardInterrupt: ignored